# Quick Interface Instructions:
* Run the cells below sequentially
* Click a cell to edit
* ** Shift-enter ** to run a cell

# Import useful tools

In [ ]:
# Be patient, this may take a little while
# If you see "In [*]" to the left of this cell, it means it is busy computing, just wait a bit longer
import os, imars3d, numpy as np, glob, time, pickle as pkl
from imars3d import jnbui as ui
from imars3d.ImageFile import ImageFile

In [ ]:
# Be patient, this may take a little while too
# %matplotlib notebook
%matplotlib inline
from matplotlib import pyplot as plt

In [ ]:
import ipywidgets as ipyw
from IPython.display import display, HTML, clear_output

# Configuration Wizard
The following inputs work:
* IPTS number:    15518
* Scan name:      derek_inj
* CT signature:   Derek_injec
* DF/OB files:    all files under df/ob dir

In [ ]:
config = ui.config()
ui.WizardPanel(ui.IPTSpanel(config))

## Change directory

In [ ]:
%mkdir -p {config.outdir}
%cd {config.outdir}

## Save configuration

In [ ]:
pkl.dump(config, open('recon-config.pkl', 'wb'))

## Print configuration

In [ ]:
for k, v in config.__dict__.items():
    if k.startswith('_'): continue
    sv = str(v)
    if len(sv) > 60:
        sv = sv[:50] + '...'
    print "{0:20}{1:<}".format(k,sv)

# Create CT data object

In [ ]:
# import the CT class
from imars3d.CT import CT

In [ ]:
# ask for help
# CT?

In [ ]:
ct = CT(config.datadir, 
        CT_subdir="ct_scans/%s" % config.ct_subdir, CT_identifier=config.ct_sig,
        workdir=config.workdir, outdir=config.outdir,
        ob_files=config.ob_files, df_files=config.df_files,
        # ob_identifier=ob_sig, df_identifier=df_sig
       )

# Preprocess

In [ ]:
%%time
ppd = ct.preprocess()

# Find region of interests (ROI)

In [ ]:
# Inspect the pre-processed image files 
# print ppd

In [ ]:
# Inspect one image. Change the index below to see a different image
index = 0
ppd_img0 = ppd.getData(ppd.identifiers[index])
plt.pcolormesh(ppd_img0, cmap='gray')
plt.colorbar()
# adjust the following to make better plot
# plt.clim(0, 30)

In [ ]:
# Randomly pick some preprocessed files and average them
samples = np.random.choice(len(ppd.identifiers), 10)
data = 0
for sample in samples:
    data1 = ppd.getData(ppd.identifiers[sample])
    data += data1
    continue
data/=len(samples)

# then plot
plt.pcolormesh(data, cmap='gray')
plt.colorbar()
# adjust the following to make a better plot
# plt.clim(0, 30)

Please look at the image and find the ROI.
Write down the bounding box for the ROI.

** For this sample, we can define the ROI as xmin=500, ymin=0, xmax=1600, ymax=2047 **

## Run reconstruction

In [ ]:
%%time
# Now run the reconstruction. 
# Make sure to modify the crop_window and use the numbers obtained from above
# The reconstruction will take 5-60 minutes.
ct.recon(crop_window=(500,0,1600,2047))

# Check output

## The reconstruction output directory

In [ ]:
print config.outdir

## The reconstruction output files

In [ ]:
!ls -l {config.outdir} | head -n 10

## The temporay work directory

In [ ]:
print config.workdir

In [ ]:
!ls -l {config.workdir}

** Please note that the temporary working directory will be pruned after a certain period of time. If you want to retain those files, please copy them to your home directory or the experiment IPTS folder **

## Example of temp output - sinograms

In [ ]:
!ls -l {config.workdir}/sinogram | head -n 10

# Visualization

ImageJ. To start, open a terminal and type
```
$ /HFIR/CG1DImaging/shared/Fiji.app/ImageJ-linux64
```